In [ ]:
### A showcase for foursight audit of status check
### 1) get all main node (exp set, exp, publication, etc) uuids
### 2) loop thorugh them, fetch all related items
### 3) on the collection of items, run the status comparison, main node being the reference

from dcicutils import ff_utils
from functions.notebook_functions import *
import time
time1 = time.time()

env = 'data'
my_auth = get_key('koray_data')
schema_name = get_schema_names(my_auth) 

sets_url = '/search/?status=in+review+by+lab&type=ExperimentSetReplicate&field=uuid'
all_sets = [i['uuid'] for i in ff_utils.search_metadata(sets_url, key = my_auth)]
print(len(sets_resp), 'exps found')

def get_time(time_1):
    time2 = time.time()
    diff = time2-time_1
    if diff>60: 
        diff_p = str((diff)/60)[:3]+' min' 
    else: 
        diff_p = str(diff)[:3] + ' sec'
    return diff_p

print(get_time(time1))

In [ ]:
def status_check(store):
    # this status dic is not meant to be accurate, just for testing.
    # I assume the main item to be an experiment set, but can be adjusted to accept any type.
    status_dic = {
        'deleted':0,
        'replaced': 0,
        'in review by lab': 1,
        'in review by project': 1,
        'uploaded': 1,
        'released to project': 2,
        'released': 3,
        'current':3
    }
    main_item = store['experiment_set_replicate']
    assert len(main_item) == 1 # did the store collected more then one items
    main_status = main_item[0]['status']
    status_report = []
    for a_type in store:
        type_status = [(i['@id'], i['status']) for i in store[a_type] if status_dic[i['status']] < status_dic[main_status]]
        
        status_report.extend(type_status)
    return status_report


In [ ]:
time1 = time.time()

for a_set in all_sets:
    time_set_1 = time.time()
    store, uuids = record_object_es([a_set], my_auth, schema_name, store_frame='object')
    print(get_time(time_set_1), a_set, len(uuids))
    statuses = status_check(store)
    print(len(statuses))
    print(statuses)
    print()
    
print('total', get_time(time1))